# Hyperparameter Grid Search 

We find the optimal hyperparameters via grid search, which is just iterating through all combinations of hyperparameters using the same model.

In [65]:
# basic imports
from pathlib import Path
import numpy as np
import pandas as pd

#machine learning imports
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score, precision_score
from libsvm.svmutil import *
from libsvm.svm import *

## Reading in CSV Data

In [66]:
#Get csv dataset from data folder and read it
data_path = "../data"
output_path = "./output"

csv_name = 'frequencies.csv'
csv_path = Path(data_path, csv_name)

train_file_name = Path(output_path, "freq_train") 
test_file_name = Path(output_path, "freq_test") 

df = pd.read_csv(csv_path)

## Converting data to LibSVM Format

In [67]:
#remove index as its not useful and get outcomes from hasHospitalizations
df = df.drop("Unnamed: 0", axis = 1)
y = df['hasHospitilization']
x = df.drop(['hasHospitilization'], axis=1)

#split into training and testing data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

#convert outcomes to LibSVM format: 1 -> -1 (anomaly) and 0 -> 1(normal)
convert_outcome = lambda x: -2*x+1
y_train_convert = convert_outcome(y_train)
y_test_convert = convert_outcome(y_test)

#scale data from 0 to 1 to avoid domination of one factor
scaler = MinMaxScaler(copy = False)
scaler.fit_transform(x_train)
scaler.transform(x_test)

#LibSVM requires input vectors/matrices to be list type
input_train = x_train.values.tolist()
input_test = x_test.values.tolist()
outcomes_train = y_train_convert.to_list()
outcomes_test = y_test_convert.to_list()

# Grid Searching the Best Set of Hyperparameters

In [68]:
train_prob = svm_problem(outcomes_train,input_train)

#params:
#  -s:   Choose OCSVM model. 2 means One Class SVM
#  -b:   Choose to predict probability. 1 means predict probability
#  -t:   Chooses kernel type (linear, poly, radial, sigmoid, and precomputed kernel)
#  -n:   nu hyperparameter, upper limit of incorrect labels, lower means less tolerance
#  -g:   gamma hyperparameter, determines similarity required to be in same class, higher means more curvature
#  -h:   Use shrinking heuristic or not

#Commonly used hyperparameters
kernel_candidates = ["Linear", "Polynomial", "Radial", "Sigmoid"]
nu_candidates = [0.0001, 0.001, 0.01, 0.1]
gamma_candidates = [0.001, 0.0025, 0.005, 0.01, 0.025, 0.05, 0.1]

#instantiate k*nu*g size arrays to hold performance scores for each run
prec_scores = [[[0 for x in range(len(gamma_candidates))] for y in range(len(nu_candidates))] for z in range(len(kernel_candidates))]
recall_scores = [[[0 for x in range(len(gamma_candidates))] for y in range(len(nu_candidates))] for z in range(len(kernel_candidates))]
f1_scores = [[[0 for x in range(len(gamma_candidates))] for y in range(len(nu_candidates))] for z in range(len(kernel_candidates))]

#run each combination of hyperparameters, recording the stats for each one
for k_index, kernel_type in enumerate(kernel_candidates):
   for nu_index, nu in enumerate(nu_candidates):
      for g_index, gamma in enumerate(gamma_candidates):
         #set new parameters
         params = svm_parameter(f'-s 2 -b 1 -t {k_index} -h 0 -n {nu} -g {gamma}')

         #train and save model
         model = svm_train(train_prob, params)

         #ignore the accuracy rating, not meant for OCSVMs
         #can't figure out how to remove accuracy rating print
         p_labels, p_acc, p_vals = svm_predict(outcomes_test, input_test, model)

         #Convert predictions into true/false
         pred_outcomes = np.array(p_labels) < 0
         actual_outcomes = np.array(y_test_convert) < 0

         #Determine performance metrics and record them into their corresponding matrix
         prec_score = precision_score(actual_outcomes, pred_outcomes, average = "binary")
         rec_score = recall_score(actual_outcomes, pred_outcomes, average = "binary")
         f_score = f1_score(actual_outcomes, pred_outcomes, average = "binary")

         prec_scores[k_index][nu_index][g_index] = prec_score
         recall_scores[k_index][nu_index][g_index] = rec_score
         f1_scores[k_index][nu_index][g_index] = f_score

Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4868% (9305/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4868% (93

C:\Users\Brendan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy = 26.3124% (2461/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 26.291% (2459/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 26.291% (2459/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 28.3438% (2651/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 16.5081% (1544/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.5189% (9308/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 26.569% (2485/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 85.7479% (8020/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 85.7479% (8020/9353) (classification)
Model supports probability estimates, but 

C:\Users\Brendan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Brendan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy = 99.5937% (9315/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 98.5352% (9216/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.4547% (9302/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 60.3443% (5644/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 96.8673% (9060/9353) (classification)
Model supports probability estimates, but disabled in predicton.
Accuracy = 99.3585% (9293/9353) (classification)
Model supports probability estimates, b

# Results

In [69]:
# takes a 2D array and formats it into a Pandas Dataframe to give labels and pretty print
def format_table(results, row_labels, col_labels):
   df = pd.DataFrame(results)
   df.columns = col_labels
   df.index = row_labels
   print(df)

pd.set_option("display.precision", 4)

for k_index, kernel_type in enumerate(kernel_candidates):
   p = prec_scores[k_index]
   r = recall_scores[k_index]
   f1 = f1_scores[k_index]

   print(kernel_type, "Kernel")
   print("Precision Scores")
   format_table(p, nu_candidates, gamma_candidates)
   print("Recall Scores")
   format_table(r, nu_candidates, gamma_candidates)
   print("F1 Scores")
   format_table(f1,  nu_candidates, gamma_candidates)
   print()

#Find the best F1 score and what hyperparams caused it
best_f1_score = np.array(f1_scores).max()
best_params = np.where(f1_scores == best_f1_score)
best_kernel_idx, best_nu_idx, best_gamma_idx = best_params[0][0], best_params[1][0], best_params[2][0]

print("Highest F1 Score: ", best_f1_score)
print("Using parameters: ")
print("Kernel Type:", kernel_candidates[best_kernel_idx])
print("Nu Parameter:", nu_candidates[best_nu_idx])
print("Gamma Parameter", gamma_candidates[best_gamma_idx])

Linear Kernel
Precision Scores
        0.0010  0.0025  0.0050  0.0100  0.0250  0.0500  0.1000
0.0001  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
0.0010  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
0.0100  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
0.1000  0.0054  0.0054  0.0054  0.0054  0.0054  0.0054  0.0054
Recall Scores
        0.0010  0.0025  0.0050  0.0100  0.0250  0.0500  0.1000
0.0001  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
0.0010  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
0.0100  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
0.1000  0.1316  0.1316  0.1316  0.1316  0.1316  0.1316  0.1316
F1 Scores
        0.0010  0.0025  0.0050  0.0100  0.0250  0.0500  0.1000
0.0001  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
0.0010  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
0.0100  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
0.1000  0.0103  0.0103  0.0103  0.0103  0.0103  0.0103  0.0103
